## LSTM-CNN Text Classification with Keras

In [1]:
%run Setup.ipynb

[i] Loaded Parameters:
 40000 30 0.2 50 
 dataset/glove/glove.twitter.27B.50d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[i] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30692 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 4)
[+] Number of entries in each category:
[+] Training:
 [ 7690. 10981. 12192.  1137.]
[+] Validation:
 [1954. 2643. 3107.  296.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.50d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

### RNN Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(12,dropout=0.4,recurrent_dropout=0.4,return_sequences=True))(embedded_sequences)
l_cov1= Conv1D(50, 3, activation='relu',kernel_regularizer=regularizers.l2(0.02))(l_lstm1)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_drop1 = Dropout(0.4)(l_pool1)
#l_cov2 = Conv1D(32, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_drop1)
#l_pool2 = MaxPooling1D(2)(l_cov2)
#l_drop2 = Dropout(0.3)(l_pool2)
l_flat = Flatten()(l_drop1)
l_dense = Dense(25, activation='relu')(l_flat)
preds = Dense(4, activation='softmax')(l_dense)

In [4]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

In [5]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc', lr_metric])

In [6]:
def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.00001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)
    
def initial_boost(epoch):
    if epoch==0: return float(6.0)
    else: return float(1.0)
        
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=5, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#lr_schedule = callbacks.LearningRateScheduler(step_cyclic)
lr_schedule = callbacks.LearningRateScheduler(step_cyclic)

In [7]:
model.summary()
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=64,
          callbacks=[tensorboard, model_checkpoints, lr_schedule])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 50)            1534650   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 30, 24)            6048      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 28, 50)            3650      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 14, 50)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 50)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 700)               0         
__________

Epoch 32/200
Epoch 32 - learning_rate 0.9996900960702182
32000/32000 [==============================] - 28s 880us/step - loss: 0.9740 - acc: 0.5954 - lr: 0.9997 - val_loss: 0.9788 - val_acc: 0.5907 - val_lr: 0.9997
Epoch 33/200
Epoch 33 - learning_rate 0.9996801023672424
32000/32000 [==============================] - 29s 894us/step - loss: 0.9743 - acc: 0.5981 - lr: 0.9997 - val_loss: 0.9853 - val_acc: 0.5941 - val_lr: 0.9997
Epoch 34/200
Epoch 34 - learning_rate 9.99670108864075
32000/32000 [==============================] - 28s 889us/step - loss: 1.0285 - acc: 0.5725 - lr: 9.9967 - val_loss: 1.0265 - val_acc: 0.5725 - val_lr: 9.9967
Epoch 35/200
Epoch 35 - learning_rate 0.9996601155607093
32000/32000 [==============================] - 28s 884us/step - loss: 0.9859 - acc: 0.5980 - lr: 0.9997 - val_loss: 0.9952 - val_acc: 0.5883 - val_lr: 0.9997
Epoch 36/200
Epoch 36 - learning_rate 0.9996501224571399
32000/32000 [==============================] - 29s 893us/step - loss: 0.9749 - acc: 0

32000/32000 [==============================] - 29s 912us/step - loss: 0.9246 - acc: 0.6310 - lr: 0.9993 - val_loss: 0.9931 - val_acc: 0.5863 - val_lr: 0.9993
Epoch 71/200
Epoch 71 - learning_rate 0.99930048965724
32000/32000 [==============================] - 31s 954us/step - loss: 0.9189 - acc: 0.6338 - lr: 0.9993 - val_loss: 0.9980 - val_acc: 0.5871 - val_lr: 0.9993
Epoch 72/200
Epoch 72 - learning_rate 0.999290503742343
32000/32000 [==============================] - 28s 889us/step - loss: 0.9156 - acc: 0.6380 - lr: 0.9993 - val_loss: 0.9919 - val_acc: 0.5830 - val_lr: 0.9993
Epoch 73/200
Epoch 73 - learning_rate 0.9992805180270204
32000/32000 [==============================] - 29s 891us/step - loss: 0.9137 - acc: 0.6363 - lr: 0.9993 - val_loss: 1.0009 - val_acc: 0.5841 - val_lr: 0.9993
Epoch 74/200
Epoch 74 - learning_rate 0.9992705325112669
32000/32000 [==============================] - 27s 859us/step - loss: 0.9134 - acc: 0.6375 - lr: 0.9993 - val_loss: 0.9949 - val_acc: 0.5846 - 

32000/32000 [==============================] - 28s 888us/step - loss: 0.8623 - acc: 0.6700 - lr: 0.9989 - val_loss: 1.0122 - val_acc: 0.5757 - val_lr: 0.9989
Epoch 109/200
Epoch 109 - learning_rate 0.9989211651416471
32000/32000 [==============================] - 28s 864us/step - loss: 0.8603 - acc: 0.6670 - lr: 0.9989 - val_loss: 1.0226 - val_acc: 0.5760 - val_lr: 0.9989
Epoch 110/200
Epoch 110 - learning_rate 0.998911186806381
32000/32000 [==============================] - 27s 847us/step - loss: 0.8550 - acc: 0.6687 - lr: 0.9989 - val_loss: 1.0303 - val_acc: 0.5733 - val_lr: 0.9989
Epoch 111/200
Epoch 111 - learning_rate 0.9989012086704624
32000/32000 [==============================] - 28s 880us/step - loss: 0.8568 - acc: 0.6697 - lr: 0.9989 - val_loss: 1.0161 - val_acc: 0.5767 - val_lr: 0.9989
Epoch 112/200
Epoch 112 - learning_rate 0.9988912307338854
32000/32000 [==============================] - 28s 888us/step - loss: 0.8545 - acc: 0.6692 - lr: 0.9989 - val_loss: 1.0173 - val_acc:

32000/32000 [==============================] - 28s 885us/step - loss: 0.7986 - acc: 0.7019 - lr: 0.9986 - val_loss: 1.0421 - val_acc: 0.5736 - val_lr: 0.9986
Epoch 147/200
Epoch 147 - learning_rate 0.9985421284924011
32000/32000 [==============================] - 28s 887us/step - loss: 0.8006 - acc: 0.6994 - lr: 0.9985 - val_loss: 1.0541 - val_acc: 0.5686 - val_lr: 0.9985
Epoch 148/200
Epoch 148 - learning_rate 0.9985321577281395
32000/32000 [==============================] - 28s 889us/step - loss: 0.7967 - acc: 0.7025 - lr: 0.9985 - val_loss: 1.0590 - val_acc: 0.5691 - val_lr: 0.9985
Epoch 149/200
Epoch 149 - learning_rate 0.9985221871629988
32000/32000 [==============================] - 28s 863us/step - loss: 0.7986 - acc: 0.7020 - lr: 0.9985 - val_loss: 1.0390 - val_acc: 0.5683 - val_lr: 0.9985
Epoch 150/200
Epoch 150 - learning_rate 0.9985122167969726
32000/32000 [==============================] - 27s 848us/step - loss: 0.7946 - acc: 0.7035 - lr: 0.9985 - val_loss: 1.0473 - val_acc

32000/32000 [==============================] - 28s 883us/step - loss: 0.7375 - acc: 0.7320 - lr: 0.9982 - val_loss: 1.0751 - val_acc: 0.5633 - val_lr: 0.9982
Epoch 185/200
Epoch 185 - learning_rate 0.9981633793819372
32000/32000 [==============================] - 27s 854us/step - loss: 0.7394 - acc: 0.7323 - lr: 0.9982 - val_loss: 1.0780 - val_acc: 0.5601 - val_lr: 0.9982
Epoch 186/200
Epoch 186 - learning_rate 0.9981534161800669
32000/32000 [==============================] - 27s 846us/step - loss: 0.7354 - acc: 0.7317 - lr: 0.9982 - val_loss: 1.0920 - val_acc: 0.5594 - val_lr: 0.9982
Epoch 187/200
Epoch 187 - learning_rate 0.9981434531770906
32000/32000 [==============================] - 28s 886us/step - loss: 0.7331 - acc: 0.7324 - lr: 0.9981 - val_loss: 1.0936 - val_acc: 0.5607 - val_lr: 0.9981
Epoch 188/200
Epoch 188 - learning_rate 0.9981334903730025
32000/32000 [==============================] - 28s 888us/step - loss: 0.7337 - acc: 0.7348 - lr: 0.9981 - val_loss: 1.0921 - val_acc

In [8]:
model.save('ltsm-c.h5')